https://livebook.manning.com/book/deep-learning-with-python/chapter-8/32

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

Download training dataset.

In [ ]:
path = keras.utils.get_file('nietzsche.txt',
                           origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Vectorizing sequences of characters.

In [ ]:
maxlen = 60
step = 3

In [ ]:
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences: ', len(sentences))

In [ ]:
chars = sorted(list(set(text)))
print('Unique characters: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

In [ ]:
print('Vectorization')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
x.shape

In [ ]:
from keras import layers

In [ ]:
(maxlen, len(chars))

In [ ]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars)),use_bias=False))
model.add(layers.Dense(len(chars), activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probax)

In [ ]:
import random
import sys

Text-generation loop.

In [ ]:
for epoch in range(1, 60):
    print('epoch:', epoch)
    model.fit(x,y,batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen -1 )
    generated_text = text[start_index: start_index + maxlen]
    print('Generating with seed: ', generated_text)
    for temperature in [.2,.5,1.0, 1.2]:
        print('temperature: ', temperature)
        print(generated_text)
        
        for i in range(400):
            samples = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            print(generated_text)